# 📄 Resume-to-O*NET Semantic Similarity Pipeline

This notebook computes semantic similarity between **resume text** and multiple **O*NET categories** (Knowledge, Skills, Abilities, Work Activities) using a transformer-based embedding model. The goal is to identify relevant O*NET concepts mentioned in resumes based on cosine similarity of noun phrases extracted from the text.

---

## 🚀 Features

- Loads resumes from a SQLite database with user annotations (`rating = 5`)
- Processes multiple O*NET CSV datasets from a directory
- Uses noun phrase extraction to analyze resume content
- Computes semantic similarity using sentence embeddings
- Saves similarity results to CSV files for each O*NET category

---

## 📦 Input Files

### SQLite Database

The database must include:
- `resumes` table with columns: `id`, `resume_text`
- `annotations` table with: `resume_id`, `rating`
- `predicted_jobs` table with: `resume_id`, `job_title`

Only resumes with `rating = 5` will be processed.

### O*NET CSV Files

Each CSV file should contain:
- `job_title`: the job name/title
- `knowledge_entity`: the concept, skill, or activity
- `data_value`: importance or level (numeric)

These files are automatically processed one by one.

---

## 🧠 How It Works

1. Load resumes with `rating = 5` from the SQLite database.
2. Loop over each O*NET CSV file (e.g., Knowledge, Skills, etc.).
3. For each resume:
   - Filter relevant O*NET entities by matching job titles.
   - Extract noun phrases from the resume text.
   - Generate sentence embeddings for both noun phrases and O*NET entities.
   - Compute cosine similarity between each pair.
   - Keep matches with similarity ≥ 0.65.
4. Save the similarity results into a separate CSV file for each category.

---

## 🧾 Output Format

Each output file contains:

| Column           | Description                                |
|------------------|--------------------------------------------|
| resume_id        | Resume ID                                  |
| job_title        | Job title associated with the resume       |
| noun_phrase      | Extracted noun phrase from resume          |
| knowledge_entity | Matching O*NET entity                      |
| similarity_score | Cosine similarity score (0 to 1)           |
| data_value       | Importance or level from O*NET (numeric)   |

---

## ✅ Sample Output



In [5]:
import os
import sqlite3
import pandas as pd
import logging
from textblob import TextBlob
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
from tqdm import tqdm
import os

In [2]:
# Setup
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Device detection
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Output directory
output_dir = "../data/similarity_outputs"
os.makedirs(output_dir, exist_ok=True)

# Function: Compute similarity between resume and O*NET category (e.g., knowledge)
def compute_resume_similarity(resume_row, df_onet, model, threshold=0.65, batch_size=32):
    similarity_results = []

    resume_id = resume_row["resume_id"]
    resume_text = resume_row["resume_text"]
    job_title = resume_row["job_title"]

    # Match job title
    df_knowledge = df_onet[df_onet["job_title"].str.contains(job_title, case=False, na=False, regex=True)]

    if df_knowledge.empty:
        logging.warning(f"No entities found for job: {job_title} (Resume ID: {resume_id})")
        return similarity_results

    # Extract noun phrases
    blob = TextBlob(resume_text)
    noun_phrases = list(set(blob.noun_phrases))
    if not noun_phrases:
        logging.warning(f"No noun phrases found in resume ID {resume_id}")
        return similarity_results

    # Embeddings
    resume_embeddings = model.encode(noun_phrases, convert_to_numpy=True, batch_size=batch_size)
    knowledge_entities = df_knowledge["knowledge_entity"].drop_duplicates().tolist()
    knowledge_embeddings = model.encode(knowledge_entities, convert_to_numpy=True, batch_size=batch_size)

    # Cosine similarity
    similarity_matrix = cosine_similarity(resume_embeddings, knowledge_embeddings)

    for i, noun_phrase in enumerate(noun_phrases):
        for j, knowledge_entity in enumerate(knowledge_entities):
            score = similarity_matrix[i, j]
            if score >= threshold:
                data_value = df_knowledge[df_knowledge["knowledge_entity"] == knowledge_entity]["data_value"].values[0]
                similarity_results.append({
                    "resume_id": resume_id,
                    "job_title": job_title,
                    "noun_phrase": noun_phrase,
                    "knowledge_entity": knowledge_entity,
                    "similarity_score": score,
                    "data_value": data_value
                })

    return similarity_results



INFO: Using device: cuda:0
INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [7]:
# Step 1: Load resumes from DB
db_path = "../data/annotations_scenario_1/annotations_scenario_1.db"
conn = sqlite3.connect(db_path)
query_resumes = """
SELECT r.id AS resume_id, r.resume_text, pj.job_title
FROM resumes r
JOIN annotations a ON r.id = a.resume_id
JOIN predicted_jobs pj ON r.id = pj.resume_id
WHERE a.rating >= 3;
"""
df_resumes = pd.read_sql_query(query_resumes, conn)
conn.close()

# Step 2: Loop over each CSV in the o_net_data directory
onet_dir = "../data/o_net_files"
for filename in os.listdir(onet_dir):
    if filename.endswith(".csv"):
        category_name = os.path.splitext(filename)[0].lower()  # e.g., "Knowledge.csv" → "knowledge"
        logging.info(f"🔍 Processing category: {category_name}")

        df_onet = pd.read_csv(os.path.join(onet_dir, filename))

        # Step 3: Compute similarities
        all_similarity_results = []
        for _, row in tqdm(df_resumes.iterrows(), total=len(df_resumes), desc=f"Matching for {category_name}"):
            results = compute_resume_similarity(row, df_onet, model)
            all_similarity_results.extend(results)

        # Step 4: Save results
        df_similarity = pd.DataFrame(all_similarity_results).drop_duplicates()
        output_path = os.path.join(output_dir, f"resume_{category_name}_similarity_matrix.csv")
        df_similarity.to_csv(output_path, index=False)

        logging.info(f"✅ Saved: {output_path}")

logging.info("🎉 Done computing similarity for all O*NET categories.")


INFO: 🔍 Processing category: abilities
Matching for abilities:   0%|          | 0/2825 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Matching for abilities:   0%|          | 1/2825 [00:02<1:53:49,  2.42s/it]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Matching for knowledge:  57%|█████▋    | 1605/2825 [02:40<02:45,  7.38it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Matching for knowledge:  57%|█████▋    | 1605/2825 [02:40<02:01, 10.03it/s]


KeyboardInterrupt: 